<h1><center> What's Policy Learning?

**Policy** is a function which maps current observation to an action. It is more direct way for an Agent to learn *good behaviour* by acting in the environment. The other way, **Q Learning**, learns a quantified value for being in a particular state $s$ and performing a particular action $a$. That helps in exploration, PL doesn't have that much of foresightedness since it doesn't consider long term rewards in choosing policy *(No Bellman equation)*. 

So, Policy Learning may be used for environments where reward is based on immediate action *(non-hysterical)*. 

$$\pi(s) \rightarrow a$$